# Laboratório 2: Criando uma rede neural simples com o keras

Bem vindo ao laboratório 2 do Mini curso de TensorFlow, neste tutorial vamos aprender a classificar imagens de roupas como tênis e camisetas utilizando o TensorFlow 2 e o keras.

<img src="imgs/tf_logo_social.png" width="500">

O Keras (tf.keras) é uma API de alto nível para o TensorFlow que nos pertmite construir e treinar modelos de deep learning de ma forma rápida e simplificada. Além de poupar a escrita de muitas linhas de código, o tf.keras possui algumas vantagens que tornam essa API indicada para quem está começando e não deseja se aprofundar na parte matemática de início:

* Interface otimizada e de uso simplificado para casos simples.
* Permite prototipagem fácil e rápida.
* Suporta redes convolucionais e redes recorrentes, bem como combinações das duas.
* Funciona perfeitamente na CPU e GPU.

O objetivo deste laboratório será a construção de uma rede neural que permita classificar imagens de roupas como tênis e camisetas de forma simplificada e com poucas linhas de código, no final será possível visualizar a classificação efetuada pela rede e compará-la com a classificação real de cada ítem. Utilizaremos como base de dados a Fashion MNIST que contém cerca de 70,000 imagens em tons de cinza em 10 categorias, cada imagem mostra artigos individuais de roupas com baixa resolução (28 por 28 pixels), como podemos observar abaixo:

<img src="imgs/fashion-mnist-sprite.png" width="500">

A Fashion MNIST tem como intenção substituir a clássica base de dados MNIST que era frequentemente utilizada como "Hello, World" em programas de aprendizado de máquina (machine learning) voltados para visão computacional. 

##  Importando as bibliotecas necessárias para o projeto

Para realizarmos a construção da nossa rede neural com o Keras, precisamos importar algumas bibliotecas essenciais para a construção da nossa rede e algumas outras bibliotecas que nos  ajudarão a ter uma melhor visualização e manipulação dos dados. O código abaixo realiza as importações necessárias, não se preocupe pois as respectivas bibliotecas já se encontram devidamente instaladas:

In [5]:
# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras

# Bibliotecas auxiliares
import numpy as np
import matplotlib.pyplot as plt

# Verificamos se a importação foi realizada
print(tf.__version__) 

1.14.0
